In [1]:
import os

# ✅ Works in BOTH Jupyter and .py files
if "__file__" in globals():
    PROJECT_ROOT = os.path.dirname(os.path.abspath(__file__))
else:
    PROJECT_ROOT = os.getcwd()   # Jupyter fallback

print("✅ Project root:", PROJECT_ROOT)

# ✅ Go one level up
PARENT_ROOT = os.path.dirname(PROJECT_ROOT)

# ✅ Enter the data folder
DATA_ROOT = os.path.join(PARENT_ROOT, "data")

# ✅ Clean normalized path
DATA_ROOT = os.path.abspath(DATA_ROOT)

print("✅ Data root:", DATA_ROOT)

✅ Project root: /home/ritwikg3/CS-441-Applied-ML-Project/notebooks
✅ Data root: /home/ritwikg3/CS-441-Applied-ML-Project/data


In [ ]:
# data_url = "http://hog.ee.columbia.edu/craffel/lmd/lmd_matched.tar.gz"
# target_path = os.path.join(DATA_ROOT, "lmd_matched.tar.gz")

# # Download using wget via shell
# !wget -O "{target_path}" "{data_url}"

In [ ]:
# # Extract into the data folder
# !tar -xvzf "{target_path}" -C "{DATA_ROOT}"

In [3]:
import os
import json
import shutil
import random
import re

# ... your PROJECT_ROOT / DATA_ROOT setup above ...

MIDI_ROOT   = os.path.join(DATA_ROOT, "lmd_matched")
LASTFM_ROOT = os.path.join(DATA_ROOT, "lastfm")
MD5_JSON    = os.path.join(DATA_ROOT, "md5_to_paths.json")
OUTPUT_ROOT = os.path.join(DATA_ROOT, "mini_dataset")

if os.path.exists(OUTPUT_ROOT):
    print(f"Removing old data in: {OUTPUT_ROOT}")
    shutil.rmtree(OUTPUT_ROOT)

os.makedirs(OUTPUT_ROOT)
print(f"Folder ready: {OUTPUT_ROOT}")

CATEGORIES = ["pop", "classic", "guitar", "electronic", "piano",
              "hip-hop", "rock", "rnb", "dance", "folk"]
NUM_PER_CATEGORY = 100

with open(MD5_JSON, "r") as f:
    md5_to_paths = json.load(f)

print(f"Loaded MD5 mapping for {len(md5_to_paths)} keys.")

saved_dict = {category: [] for category in CATEGORIES}

# ✅ GLOBAL set of songs already used across ALL categories
used_md5_keys = set()

for category in CATEGORIES:
    print(f"\nProcessing category: {category}")

    category_file = os.path.join(LASTFM_ROOT, f"id_list_{category}.txt")
    if not os.path.exists(category_file):
        print(f"⚠ File not found: {category_file}")
        continue

    with open(category_file, "r") as f:
        track_ids = [line.strip() for line in f.readlines()]

    copied_count = 0

    for track_id in track_ids:
        if copied_count >= NUM_PER_CATEGORY:
            break

        folder_structure = os.path.join(
            MIDI_ROOT, track_id[2], track_id[3], track_id[4], track_id
        )
        if not os.path.exists(folder_structure):
            print(f"⚠ Track folder missing: {folder_structure}")
            continue

        track_folder = folder_structure

        midi_files = [f for f in os.listdir(track_folder) if f.lower().endswith(".mid")]
        if not midi_files:
            print(f"⚠ No MIDI file in {track_folder}")
            continue

        hashed_midi = midi_files[0]
        md5_key = os.path.splitext(hashed_midi)[0]

        # ✅ SKIP if this song has already been used in any previous category
        if md5_key in used_md5_keys:
            print(f"🔁 Skipping {md5_key}: already used in another category")
            continue

        if md5_key not in md5_to_paths:
            print(f"⚠ MD5 not found in JSON: {hashed_midi}")
            continue

        new_file_name = md5_to_paths[md5_key][0]
        src_path = os.path.join(track_folder, hashed_midi)

        dest_folder = os.path.join(OUTPUT_ROOT, category)
        os.makedirs(dest_folder, exist_ok=True)

        safe_name = re.sub(r'[<>:"/\\|?*]', '_', os.path.basename(new_file_name))
        safe_name = safe_name.replace("-", "_")
        dest_path = os.path.join(dest_folder, safe_name)

        try:
            shutil.copy(src_path, dest_path)
            copied_count += 1
            saved_dict[category].append(new_file_name)

            # ✅ Mark this song as used globally
            used_md5_keys.add(md5_key)

            print(f"✅ Copied and renamed {copied_count}/{NUM_PER_CATEGORY}: {dest_path}\n")
        except PermissionError:
            print(f"❌⚠ Permission denied, skipping: {dest_path}")

    print(f"Finished category {category}. Total copied (unique): {copied_count}")

print("\n✅ File count per category (unique songs across all categories):")
for category, files in saved_dict.items():
    print(f"{category}: {len(files)} files")

print(f"\n🔢 Total unique songs used: {len(used_md5_keys)}")

Removing old data in: /home/ritwikg3/CS-441-Applied-ML-Project/data/mini_dataset
Folder ready: /home/ritwikg3/CS-441-Applied-ML-Project/data/mini_dataset
Loaded MD5 mapping for 178561 keys.

Processing category: pop
✅ Copied and renamed 1/100: /home/ritwikg3/CS-441-Applied-ML-Project/data/mini_dataset/pop/Bryan Adams _ Dream on.mid

✅ Copied and renamed 2/100: /home/ritwikg3/CS-441-Applied-ML-Project/data/mini_dataset/pop/michael_buble_havent_met_you_yet.mid

✅ Copied and renamed 3/100: /home/ritwikg3/CS-441-Applied-ML-Project/data/mini_dataset/pop/When the Going Gets Tough.mid

✅ Copied and renamed 4/100: /home/ritwikg3/CS-441-Applied-ML-Project/data/mini_dataset/pop/Rock_'N_Roll_Is_King.mid

✅ Copied and renamed 5/100: /home/ritwikg3/CS-441-Applied-ML-Project/data/mini_dataset/pop/Songforg.mid

✅ Copied and renamed 6/100: /home/ritwikg3/CS-441-Applied-ML-Project/data/mini_dataset/pop/Nurmitdi.mid

✅ Copied and renamed 7/100: /home/ritwikg3/CS-441-Applied-ML-Project/data/mini_dataset/

In [4]:
print("\n✅ File count per category:")
for category, files in saved_dict.items():
    print(f"{category}: {len(files)} files")



✅ File count per category:
pop: 100 files
classic: 100 files
guitar: 100 files
electronic: 100 files
piano: 100 files
hip-hop: 100 files
rock: 100 files
rnb: 100 files
dance: 100 files
folk: 100 files
